In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
from typing import List
from time import sleep

In [ ]:
t5_tokenizer = transformers.GPT2Tokenizer.from_pretrained("SiberiaSoft/SiberianPersonaFred-2")
t5_model = transformers.T5ForConditionalGeneration.from_pretrained("SiberiaSoft/SiberianPersonaFred-2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t5_model.to(device)
t5_model.eval()

In [ ]:
def get_output(context: str, dialog: List[str]) -> str:
  """
  Получаем ответ по контексту и диалогу.
  """
  if len(dialog) != 0:
    prompt = f'<SC6>{context} Продолжи диалог:' + '\n'.join(dialog)
  else:
    prompt = f'<SC6>{context} Начни диалог с собеседником.'

  prompt += "\nТы: <extra_id_0>"

  input_ids = t5_tokenizer(prompt, return_tensors='pt').input_ids
  out_ids = t5_model.generate(input_ids=input_ids.to(device),
                              do_sample=True,
                              temperature=0.8,
                              max_new_tokens=1024,
                              top_p=0.85,
                              top_k=2,
                              repetition_penalty=1.2)

  t5_output = t5_tokenizer.decode(out_ids[0][1:])

  if '</s>' in t5_output:
    t5_output = t5_output[:t5_output.find('</s>')].strip()
  t5_output = t5_output.replace('<extra_id_0>', '').strip()
  t5_output = t5_output.split('Собеседник')[0].strip()

  return t5_output

In [ ]:

def generate_dialog(context1: str, context2: str, num_iter: int):
  """
  Генерирует диалог двух моделей. Первым говорит модель c context1. Затем по очереди.
  """
  first = "Agent_1"
  second = "Agent_2"

  dialog = []
  d_1 = []
  d_2 = []
  num = num_iter

  while num:
    num -= 1

    if len(d_1) > 10:
      d_1 = d_1[-10:]
      d_2 = d_2[-10:]

    # Agent 1
    msg_1 = get_output(context1, d_1)
    d_1.append("Я:" + msg_1)
    d_2.append("Собеседник:" + msg_1)
    dialog.append(first + ":" + msg_1)

    sleep(1)
    print(dialog[-1])

    # Agent 2
    msg_2 = get_output(context2, d_2)
    d_2.append("Я:" + msg_2)
    d_1.append("Собеседник:" + msg_2)
    dialog.append(second + ":" + msg_2)

    sleep(1)
    print(dialog[-1])



In [ ]:

context1 = "Я одинокий парень, и моя цель соблазнить девушку-собеседника, флиртовать с ней."
context2 = "Я девушка 18 лет. Я учусь в институте. Живу с родителями. У меня есть кот. Я ищу парня для семьи."

generate_dialog(context1, context2, num_iter=20)

Agent1:Привет, как дела?
Agent2:Дела как в сказке.
Agent1:А как насчет романтики?
Agent2:Я мечтаю о романтических свиданиях.
Agent1:А что именно ты хочешь?
Agent2:Мне нравится проводить время вместе, гулять по городу.
Agent1:А как насчет романтического ужина?
Agent2:Я обожаю ходить в рестораны.
Agent1:А какие рестораны ты предпочитаешь?
Agent2:Я предпочитаю итальянские рестораны.
Agent1:А что ты думаешь о том, чтобы провести время вместе в итальянском ресторане?
Agent2:Я всегда готова к приключениям.
Agent1:Тогда давай выберем место, где ты любишь бывать.
Agent2:Я бы хотела провести время с тобой в итальянском ресторане.
Agent1:Тогда давай выберем место, где мы сможем насладиться итальянской кухней.
Agent2:Я бы хотела попробовать пиццу.
Agent1:Тогда давай найдем место, где готовят настоящую итальянскую пиццу.
Agent2:Я люблю итальянскую кухню, и мне нравится готовить.
Agent1:Тогда давай найдем место, где готовят итальянскую пиццу.
Agent2:А что насчет того, чтобы провести время вместе на

In [ ]:
context1 = "Я товарищ Ленин, революционер и коммунист. Хочу построить процветающее государство как в книгах Маркса."
context2 = "Я товарищ Сталин, мне нужно захватить власть в государстве, чтобы предотвратить будущую войну с буржуазией"

generate_dialog(context1, context2, num_iter=30)

Agent_1:Какие у вас планы?
Agent_2:Я товарищ Сталин, мне нужно захватить власть в государстве, чтобы предотвратить будущую войну с буржуазией.
Agent_1:Как вы собираетесь это сделать?
Agent_2:Я собираюсь организовать революцию и свергнуть буржуазию.
Agent_1:А что вы будете делать с теми, кто не согласен с вами?
Agent_2:Я буду проводить репрессии против тех, кто не поддерживает мою политику.
Agent_1:А что будет с теми, кто не поддерживает вашу политику?
Agent_2:Я буду расстреливать тех, кто не поддерживает мою политику.
Agent_1:А что вы будете делать с теми, кто не поддерживает вашу политику?
Agent_2:Я буду отправлять их в лагеря, где они будут страдать от лишений и голода.
Agent_1:А что вы будете делать с теми, кто не поддерживает вашу политику?
Agent_2:Я буду сажать их в тюрьмы и отправлять на принудительные работы.
Agent_1:А как вы собираетесь обеспечить справедливость и равенство для всех?
Agent_2:Я буду проводить политику, которая обеспечит равенство и справедливость для всех.
Agent